In [2]:
import pandas as pd
import json

In [5]:
import json
import pandas as pd

# Open the file and load the data
with open('data/pose_data.json', 'r') as file:
    data = json.load(file)  # Use json.load() for reading from a file

# Convert the loaded data into a DataFrame
df = pd.json_normalize(data, sep='_')

df

,frame,timestamp,landmarks_LEFT_SHOULDER,landmarks_RIGHT_SHOULDER,landmarks_LEFT_ELBOW,landmarks_RIGHT_ELBOW,landmarks_LEFT_WRIST,landmarks_RIGHT_WRIST,landmarks_LEFT_HIP,landmarks_RIGHT_HIP,landmarks_LEFT_KNEE,landmarks_RIGHT_KNEE,landmarks_LEFT_ANKLE,landmarks_RIGHT_ANKLE
0,9,0.30,"[-0.0538150817155838, -0.40668174624443054, -0...","[0.06761525571346283, -0.438326895236969, -0.1...","[-0.09174469113349915, -0.21799346804618835, 0...","[0.04949960485100746, -0.19054462015628815, -0...","[-0.012535757385194302, -0.059257909655570984,...","[0.07879286259412766, -0.0667775347828865, -0....","[-0.046174339950084686, -0.004013223573565483,...","[0.043879080563783646, -0.002347144763916731, ...","[0.11658654361963272, 0.17597009241580963, 0.0...","[0.06728212535381317, 0.12544633448123932, 0.0...","[0.21305391192436218, 0.4995420575141907, 0.03...","[-0.051303401589393616, 0.3669768273830414, -0..."
1,10,0.33,"[0.03386855497956276, -0.059704605489969254, -...","[0.06037702411413193, -0.2938553988933563, -0....","[0.04145700857043266, 0.15342703461647034, -0....","[0.0531480647623539, -0.18540483713150024, -0....","[0.13276788592338562, 0.39441564679145813, -0....","[-0.0027880603447556496, -0.15098345279693604,...","[-0.03174907714128494, 0.015791311860084534, -...","[0.02902970463037491, -0.02283693663775921, 0....","[-0.12097189575433731, 0.2065916359424591, -0....","[-0.05829016864299774, 0.11892367899417877, 0....","[-0.021199222654104233, 0.2557950019836426, -0...","[-0.04032081365585327, 0.17889241874217987, -0..."
2,11,0.37,"[0.01050659455358982, -0.16723693907260895, -0...","[0.08206838369369507, -0.19296038150787354, -0...","[0.0031974746380001307, 0.004731200635433197, ...","[0.06840398162603378, 0.09840293973684311, -0....","[0.02589430846273899, -0.04451915621757507, -0...","[0.011773032136261463, 0.25291553139686584, -0...","[-0.03035183809697628, 0.011522475630044937, -...","[0.02772676758468151, -0.016447218134999275, 0...","[-0.17283354699611664, -0.13578732311725616, -...","[-0.07978901267051697, -0.1670796275138855, -0...","[-0.12714079022407532, -0.2806721031665802, -0...","[-0.04331396147608757, -0.10876541584730148, -..."
3,12,0.40,"[0.0012131372932344675, -0.22261010110378265, ...","[0.08132465928792953, -0.2804141640663147, -0....","[-0.014578591100871563, -0.11539316922426224, ...","[0.05220544710755348, -0.17245396971702576, -0...","[-0.004402730148285627, -0.18524155020713806, ...","[0.025158904492855072, -0.306589812040329, 0.0...","[-0.029367836192250252, 0.01784362457692623, -...","[0.027117015793919563, -0.02534051239490509, 0...","[-0.1333128958940506, 0.18565374612808228, -0....","[-0.07749978452920914, 0.051502007991075516, 0...","[-0.14377596974372864, 0.3602980971336365, -0....","[-0.006312606390565634, 0.36765554547309875, -..."
4,13,0.43,"[-0.07598163932561874, -0.08276927471160889, -...","[0.08028367161750793, -0.18951496481895447, -0...","[-0.10421686619520187, 0.07115320861339569, -0...","[0.17557263374328613, 0.04414528235793114, 0.0...","[-0.11513210833072662, 0.23490846157073975, -0...","[0.21717317402362823, 0.20462626218795776, 0.1...","[-0.0315442830324173, 0.021015385165810585, -0...","[0.02852998860180378, -0.02731483243405819, 0....","[-0.09221181273460388, -0.07114218920469284, -...","[-0.07510575652122498, -0.3419058620929718, -0...","[-0.09737101942300797, -0.16537024080753326, -...","[-0.025274788960814476, -0.5281498432159424, -..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1338,1352,45.07,"[0.1566368043422699, 0.47103315591812134, 0.12...","[-0.15522828698158264, 0.4868610203266144, 0.0...","[0.26377350091934204, 0.3110346794128418, 0.15...","[-0.21459485590457916, 0.20666059851646423, 0....","[0.17793413996696472, 0.17887383699417114, 0.1...","[-0.328911155462265, 0.2592155933380127, 0.309...","[0.09705990552902222, -0.0005590387154370546, ...","[-0.09367547929286957, 0.0038907243870198727, ...","[0.08883839845657349, -0.37432754039764404, -0...","[-0.0407746